# <center>Zero-sum games</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass series</center>
#### <center>With python code examples</center>
© 2018-2023 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274 are acknowledged, as well as inputs from contributors listed [here](http://www.math-econ-code.org/team).

**If you reuse material from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass series. https://www.math-econ-code.org/

## Reference

* Palacios-Huerta (2003). Professionals play minimax. The Review of Economic Studies.

## Loading the data

The following data is taken from Palacios-Huerta (2003). Player 1 (kicker) takes action $i \in \{L,C, R \}$, and so does Player 2. The following table indicates the probability of a score as a matrix $\Phi_{ij}$ if Player 1 plays $i$ and Player 2 plays $j$.

In [1]:
import numpy as np
import gurobipy as grb
import scipy.sparse as spr

Phi_i_j = np.array([[50, 100, 93.8], 
                    [93.9, 60.0, 82.8], 
                    [97.6, 100, 73.2]])
Phi_i_j

array([[ 50. , 100. ,  93.8],
       [ 93.9,  60. ,  82.8],
       [ 97.6, 100. ,  73.2]])

The next table indicates the probability that player 1 takes action $i$ and that player 2 takes action $j$.

In [2]:
pi_i_j = np.array([[.158,.006,.225],[.032,.005,.034],[.208,.006,.321]])
pi_i_j

array([[0.158, 0.006, 0.225],
       [0.032, 0.005, 0.034],
       [0.208, 0.006, 0.321]])

# The zero-sum game

Consider a zero sum game.

If player $1$ plays $i\in I$ and player 2 plays $j\in J$, then:

* Payoff to player 1 is $\Phi _{ij}$

* Payoff to player 2 is $-\Phi _{ij}$

We have $\Phi _{ij}>0$.


# Pure strategies

If player 1 plays $i$, then player 2's best response is $J(i) = \arg\min_j \Phi_{ij}$.

If player 2 plays $j$, then player 1's best response is $I(j) = \arg\max_i \Phi_{ij}$.

An equilibrium in pure strategy is a pair $(i^\star,j^\star)$ such that<div>
$i^\star=I(j^\star)$ and $j^\star=J(i^\star)$.

In [3]:
class Matrix_game:
    def __init__(self,Phi_i_j):
        self.nbi,self.nbj = Phi_i_j.shape
        self.Phi_i_j = Phi_i_j

thegame = Matrix_game(Phi_i_j)

In [4]:
def BRI(self,j):
    return np.argwhere(self.Phi_i_j[:,j] == np.max(self.Phi_i_j[:,j])).flatten()

def BRJ(self,i):
    return np.argwhere(self.Phi_i_j[i,:] == np.min(self.Phi_i_j[i,:])).flatten()

Matrix_game.BRI = BRI
Matrix_game.BRJ = BRJ


In [5]:
thegame.BRI(1) ,thegame.BRJ(0) 

(array([0, 2], dtype=int64), array([0], dtype=int64))

Look for an equilibrium in pure strategy:

In [6]:
def compute_eq(self):
    return [ (i,j) for i in range(self.nbi) for j in range(self.nbj)
            if ( (i in self.BRI(j) ) and (j in self.BRJ(i) ) ) ]
Matrix_game.compute_eq = compute_eq

thegame.compute_eq()

[]

We see that in this game, there is no equilibrium in pure strategy. In fact, it is possible to show that a game has an equilibrium in pure strategy if there is no advantage in playing first in the sequential version of the game, i.e. the value of the game is the same regardless of whether player 1 plays first or second. Clearly, this is not the case here: if the kicker plays first, then the goalkeeper can move to where the kicker kicked in order to decrease the probability of a goal. If the goalkeeper plays first, then the kicker can kick where the goalkeeper did not move.

# Mixed strategies


Assume player 1 plays mixed strategy $\left( p_{i}\right) \in \Delta _{I}$,
and player 2 plays mixed strategy $\left( q_{j}\right) \in \Delta _{J}$. Then

* Expected payoff to player 1 is $\sum_{ij}p_{i}q_{j}\Phi _{ij}=p^{\top }\Phi
q $

* Payoff to player 2 is minus the former

If player 1 plays mixed strategy $p$, then player 2's best response is $Q(p) = \arg\min_{q \in \Delta_J} p^{\top }\Phi
q$.

If player 2 plays mixed strategy $q$, then player 1's best response is $P(q) = \arg\max_{p \in \Delta_I} p^{\top }\Phi
q$.

An equilibrium in pure strategy is a pair $(p^\star,q^\star)$ such that<div>
$p^\star=P(q^\star)$ and $q^\star=Q(p^\star)$.


# Equilibrium as a linear programming problem

Therefore at equilibrium, $p$ and $q$ are played so that 
\begin{equation*}
\max_{p\in \Delta _{I}}\min_{q\in \Delta _{J}}p^{\top }\Phi q
\end{equation*}
In general, we have 
\begin{equation*}
\max_{p\in \Delta _{I}}\min_{q\in \Delta _{J}}p^{\top }\Phi q\leq \min_{q\in
\Delta _{J}}\max_{p\in \Delta _{I}}p^{\top }\Phi q
\end{equation*}
but here we have in fact equality, so 
\begin{equation*}
\max_{p\in \Delta _{I}}\min_{q\in \Delta _{J}}p^{\top }\Phi q=\min_{q\in
\Delta _{J}}\max_{p\in \Delta _{I}}p^{\top }\Phi q.
\end{equation*}

Consider the problem of 
\begin{equation*}
\max_{p\in \Delta _{I}}\min_{q\in \Delta _{J}}p^{\top }\Phi q
\end{equation*}

Let us focus on the problem of player 2 if player 2 anticipates that player
1 will play $p$. This is 
\begin{eqnarray*}
\min_{q\in \Delta _{J}}p^{\top }\Phi q &=&\min_{q\in \Delta _{J}}\sum_{j\in
J}q_{j}\underset{U_{j}}{\underbrace{\left( \sum_{i}\Phi _{ij}p_{i}\right) }}
\\
&=&\min_{j\in J}\left\{ U_{j}\right\} \\
&=&\min_{j\in J}\left\{ \sum_{i}\Phi _{ij}p_{i}\right\}
\end{eqnarray*}


Recall that player 1 wants to 
\begin{equation*}
\max_{p\in \Delta _{I},U}\left\{ U:U=\min_{j\in J}\left\{ \sum_{i}\Phi
_{ij}p_{i}\right\} \right\}
\end{equation*}
but this is equivalent with 
\begin{equation*}
\max_{p\in \Delta _{I},U}\left\{ U:U\leq \min_{j\in J}\left\{ \sum_{i}\Phi
_{ij}p_{i}\right\} \right\}
\end{equation*}

Thus the program of player 1 is 
\begin{equation*}
\max_{p\in \Delta _{I},U}\left\{ U:U\leq \sum_{i}\Phi _{ij}p_{i}\text{ for
all }j\right\}
\end{equation*}
that is
\begin{eqnarray*}
V_{1}=\max_{p_{i}\geq 0,U} &&U \\
s.t.~ &&U\leq \sum_{i}\Phi _{ij}p_{i}\text{ for all }j \\
&&\sum_{i}p_{i}=1
\end{eqnarray*}


Introduce $x_{i}=p_{i}/U$. We can reformulate the previous problem after setting 
$p_{i}=x_{i}U$ by
\begin{eqnarray*}
V_{1}=\max_{x_{i}\geq 0,U} &&U \\
s.t.~ &&U\leq \sum_{i}\Phi _{ij}x_{i}U\text{ for all }j \\
&&\sum_{i}x_{i}U=1
\end{eqnarray*}
which simplifies into
\begin{eqnarray*}
V_{1}=\max_{x_{i}\geq 0,U} &&U \\
s.t.~ &&1\leq \sum_{i}\Phi _{ij}x_{i}\text{ for all }j \\
&&U=\frac{1}{\sum_{i}x_{i}}
\end{eqnarray*}
therefore
\begin{eqnarray*}
V_{1}=\max_{x_{i}\geq 0} &&\frac{1}{\sum_{i}x_{i}} \\
s.t.~ &&1\leq \sum_{i}\Phi _{ij}x_{i}\text{ for all }j
\end{eqnarray*}
that is
\begin{eqnarray*}
\frac{1}{V_{1}}=\min_{x_{i}\geq 0} &&\sum_{i}x_{i} \\
s.t.~ &&\sum_{i}\Phi _{ij}x_{i}\geq 1\text{ for all }j~\left[ y_{j}\geq 0%
\right]
\end{eqnarray*}

Recall that $x_{i}=p_{i}/U$ and $U=\frac{1}{\sum_{i}x_{i}}$.

Therefore
\begin{equation*}
p_{i}=x_{i}U=\frac{x_{i}}{\sum_{i}x_{i}}
\end{equation*}



## Player 2's problem

Consider the problem of 
\begin{equation*}
\min_{q\in \Delta _{J}}\max_{p\in \Delta _{I}}p^{\top }\Phi q
\end{equation*}
If player 1 anticipates that player 2 will play $q$, then she does 
\begin{equation*}
\max_{p\in \Delta _{I}}p^{\top }\Phi q=\max_{i\in I}\left\{ \sum_{j}\Phi
_{ij}q_{j}\right\}
\end{equation*}

Recall that player 2 wants to 
\begin{equation*}
\min_{q\in \Delta _{J},V}\left\{ V:V\geq \max_{i\in I}\left\{ \sum_{j}\Phi
_{ij}q_{j}\right\} \right\}
\end{equation*}
but this is equivalent with 
\begin{equation*}
\min_{q\in \Delta _{J},V}\left\{ V:V\geq \sum_{j}\Phi _{ij}q_{j}\right\}
\end{equation*}
that
\begin{eqnarray*}
V_{2}=\min_{q_{j}\geq 0,V} &&V \\
s.t.~ &&V\geq \sum_{j}\Phi _{ij}q_{j}\text{ for all }i \\
&&\sum_{j}q_{j}=1
\end{eqnarray*}


Introduce $y_{j}=q_{j}/V$. We can reformulate the previous problem as $
q_{j}=y_{j}V$ 
\begin{eqnarray*}
V_{2}=\min_{q_{j}\geq 0} &&\frac{1}{\sum_{j}y_{j}} \\
s.t.~ &&1\geq \sum_{j}\Phi _{ij}y_{j}\text{ for all }i
\end{eqnarray*}
that is
\begin{eqnarray*}
\frac{1}{V_{2}}=\max_{q_{j}\geq 0} &&\sum_{j}y_{j} \\
s.t.~ &&1\geq \sum_{j}\Phi _{ij}y_{j}\text{ for all }i
\end{eqnarray*}

We have 
\begin{eqnarray*}
q_{j} &=&y_{j}V=\frac{y_{j}}{\sum_{j}y_{j}} \\
V &=&\frac{1}{\sum_{j}y_{j}}
\end{eqnarray*}


# Computing the mixed strategy equilibrium using linear programming

In [7]:
def minimax_LP(self):
    model=grb.Model()
    model.Params.OutputFlag = 0
    y = model.addMVar(shape=self.nbj)
    model.setObjective(np.ones(self.nbj) @ y, grb.GRB.MAXIMIZE)
    model.addConstr(self.Phi_i_j @ y <= np.ones(self.nbi))
    model.optimize() 
    ystar = np.array(model.getAttr('x'))
    xstar = np.array(model.getAttr('pi'))
    S = 1 /  xstar.sum()
    p_i = S * xstar
    q_j = S * ystar
    return(p_i,q_j)

Matrix_game.minimax_LP = minimax_LP

In [8]:
minimax_LP(thegame)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-23


(array([0.27464503, 0.4262563 , 0.29909867]),
 array([0.26505916, 0.12247406, 0.61246679]))

## Solution using Chambolle-Pock

In [9]:
def minimax_CP(self,gap_threshold = 1e-5,max_iter = 10000):
    L1 = np.max(np.abs(self.Phi_i_j))
    sigma, tau = 1/L1, 1/L1

    p = np.ones(self.nbi) / self.nbi
    q = np.ones(self.nbi) / self.nbj
    q_prev = q.copy()

    gap = np.inf
    i=0
    while (gap >  gap_threshold) and (i < max_iter):
        q_tilde = 2*q - q_prev
        p *= np.exp(-sigma* self.Phi_i_j @ q_tilde)
        p /= p.sum()

        q_prev = q.copy()
        q *= np.exp(tau* self.Phi_i_j.T @ p)
        q /= q.sum()
        gap = np.max(self.Phi_i_j.T@p) - np.min(self.Phi_i_j@q)
        i += 1
    return(p,q,gap,i)

Matrix_game.minimax_CP = minimax_CP


In [10]:
p,q,_,_ = thegame.minimax_CP()
p,q

(array([0.274645  , 0.42625731, 0.29909769]),
 array([0.26505916, 0.1224741 , 0.61246674]))